### Create Flatfile for Running workouts

In [4]:
import pandas as pd
import numpy as np

In [5]:
### Load Files
# DimWorkouts
# FactWorkoutRecords --> Aggregate
# DimRoutes --> Aggregate
# FactBasisRecords --> Aggregate

In [6]:
DimWorkouts = pd.read_parquet('./output_files/frontend_files/DimWorkouts.parquet.gzip', engine='pyarrow')
FactWorkoutRecords = pd.read_parquet('./output_files/frontend_files/FactWorkoutRecords.parquet.gzip', engine='pyarrow')
FactRoutes = pd.read_parquet('./output_files/frontend_files/FactRoutes.parquet.gzip', engine='pyarrow')
FactWorkoutBasisRecords = pd.read_parquet('./output_files/frontend_files/FactWorkoutBasisRecords.parquet.gzip', engine='pyarrow')

### Filter for Running Workouts

In [7]:
runningWorkouts = DimWorkouts[DimWorkouts['workoutActivityType'] == 'Running']
runningFacts = FactWorkoutRecords.merge(runningWorkouts['workout_pk'], how = 'inner', on = 'workout_pk')

### Create Flatfile

#### Aggregate Values for runningFacts

In [8]:
type_list = list(runningFacts[runningFacts['workout_pk'].isna() == False]['type'].unique())
type_list

['DistanceWalkingRunning',
 'BasalEnergyBurned',
 'ActiveEnergyBurned',
 'RunningStrideLength',
 'RunningVerticalOscillation',
 'RunningGroundContactTime',
 'RunningPower',
 'RunningSpeed']

In [9]:
type_list_sum = ['DistanceWalkingRunning',
                 'BasalEnergyBurned',
                 'ActiveEnergyBurned']
type_list_avg = ['RunningStrideLength',
                 'RunningVerticalOscillation',
                 'RunningGroundContactTime',
                 'RunningPower',
                 'RunningSpeed']
type_list_sd = ['RunningStrideLength',
                'RunningVerticalOscillation',
                'RunningGroundContactTime',
                'RunningPower',
                'RunningSpeed']

In [10]:
runningFacts = runningFacts.drop(['startDate_records', 'endDate_records', 'startDate_wo', 'endDate_wo', 'unit'], axis=1)

In [11]:
columns = ['workout_pk', 'Date', 'type']

In [12]:
# Group and sum
runningFacts_sum = runningFacts[runningFacts['type'].isin(type_list_sum)].groupby(columns).sum().reset_index()
runningFacts_sum['type'] = 'SUM_' + runningFacts_sum['type']
# Pivot
runningFacts_sum = runningFacts_sum.pivot(index=['workout_pk', 'Date'], columns='type', values='value').reset_index()

# Group and avg
runningFacts_avg = runningFacts[runningFacts['type'].isin(type_list_avg)].groupby(columns).mean().reset_index()
runningFacts_avg['type'] = 'AVG_' + runningFacts_avg['type']
# Pivot
runningFacts_avg = runningFacts_avg.pivot(index=['workout_pk', 'Date'], columns='type', values='value').reset_index()

# Group and sd
runningFacts_sd = runningFacts[runningFacts['type'].isin(type_list_sd)].groupby(columns).std().reset_index()
runningFacts_sd['type'] = 'SD_' + runningFacts_sd['type']
# Pivot
runningFacts_sd = runningFacts_sd.pivot(index=['workout_pk', 'Date'], columns='type', values='value').reset_index()

merge_columns = columns
merge_columns.remove('type')

runningFactsFlat = runningFacts_sum.merge(runningFacts_avg, on = merge_columns, how = 'outer').merge(runningFacts_sd, on = merge_columns, how = 'outer')

#### Aggregate Values for DimRoutes

In [13]:
FactRoutes = FactRoutes.drop(['trkpt', 'time'], axis=1)    

In [14]:
agg_columns = ['ele', 'course', 'speed', 'hAcc', 'vAcc']

In [15]:
FactRoutes['ele'] = FactRoutes['ele'].astype(str).astype(float)
FactRoutes['course'] = FactRoutes['course'].astype(str).astype(float)
FactRoutes['speed'] = FactRoutes['speed'].astype(str).astype(float)
FactRoutes['hAcc'] = FactRoutes['hAcc'].astype(str).astype(float)
FactRoutes['vAcc'] = FactRoutes['vAcc'].astype(str).astype(float)
FactRoutes.dtypes

ele           float64
course        float64
speed         float64
hAcc          float64
vAcc          float64
route_name     object
workout_pk      int64
dtype: object

In [16]:
# Defining functions for xth percentile
def p10(x):
    return x.quantile(0.1)
def p20(x):
    return x.quantile(0.2)
def p30(x):
    return x.quantile(0.3)
def p40(x):
    return x.quantile(0.4)
def p50(x):
    return x.quantile(0.5)
def p60(x):
    return x.quantile(0.6)
def p70(x):
    return x.quantile(0.7)
def p80(x):
    return x.quantile(0.8)
def p90(x):
    return x.quantile(0.9)

aggfuncs = ['mean', 'std', p10, p20, p30, p40, p50, p60, p70, p80, p90]
runningRoutesFacts = FactRoutes.groupby(['workout_pk']).agg({'ele': aggfuncs,
                                                            'course': aggfuncs,
                                                            'speed': aggfuncs,
                                                            'hAcc': aggfuncs,
                                                            'vAcc': aggfuncs}).reset_index()

runningRoutesFacts.columns = runningRoutesFacts.columns.map('_'.join).str.strip('_')

### Aggregate Basis Records

In [17]:
runningHealthFacts = FactWorkoutBasisRecords.merge(runningWorkouts['workout_pk'], on = 'workout_pk', how = 'inner')

type_list = list(runningHealthFacts[runningHealthFacts['workout_pk'].isna() == False]['type'].unique())
type_list

['HeartRate',
 'StepCount',
 'FlightsClimbed',
 'AppleExerciseTime',
 'VO2Max',
 'HeadphoneAudioExposure',
 'AppleStandTime',
 'StairAscentSpeed',
 'StairDescentSpeed',
 'WalkingDoubleSupportPercentage',
 'WalkingSpeed',
 'WalkingStepLength',
 'WalkingAsymmetryPercentage',
 'EnvironmentalAudioExposure']

In [18]:
runningHealthFacts = runningHealthFacts[runningHealthFacts['type'] == 'HeartRate']
runningHealthFacts = runningHealthFacts.rename(columns={"value": "HeartRate"}, errors="raise")
runningHealthFactsHeartrate = runningHealthFacts.drop(['startDate_records', 'endDate_records', 'unit', 'type'], axis=1)
columns = ['workout_pk', 'Date', 'type']

In [19]:
aggfuncs = ['mean', 'std', 'min', p10, p20, p30, p40, p50, p60, p70, p80, p90, 'max']
runningHealthFactsHeartrate = runningHealthFactsHeartrate.groupby(['workout_pk']).agg({'HeartRate': aggfuncs}).reset_index()

runningHealthFactsHeartrate.columns = runningHealthFactsHeartrate.columns.map('_'.join).str.strip('_')

### Investigate Zones

In [20]:
runningHealthFacts = runningHealthFacts.sort_values(['workout_pk', 'startDate_records'])
runningHealthFacts['endDate_records_shift'] = runningHealthFacts.groupby(['workout_pk'])['endDate_records'].shift(-1)

runningHealthFacts['endDate_records_shift'] = np.where(pd.isnull(runningHealthFacts['endDate_records_shift']), runningHealthFacts['endDate_records'], runningHealthFacts['endDate_records_shift']) 

In [21]:
runningHealthFacts = runningHealthFacts.drop('endDate_records', axis = 1)

In [22]:
runningHealthFacts['duration_sec'] = (runningHealthFacts['endDate_records_shift'] - runningHealthFacts['startDate_records']).dt.total_seconds()

In [23]:
heart_ranges = [[90, 100],
                [100, 110],
                [110, 120],
                [120, 125],
                [125, 130],
                [130, 135],
                [135, 140],
                [140, 145],
                [145, 150],
                [150, 155],
                [155, 160],
                [160, 165],
                [165, 170],
                [170, 175],
                [175, 180],
                [180, 185],
                [185, 190],
                [190, 195],
                [195, 200],
                [200, 250]]

list_hr_columns = [] 

for range in heart_ranges:
    runningHealthFacts[f'HR_{range[0]}-{range[1]}'] = np.where((runningHealthFacts['HeartRate'] > range[0]) & (runningHealthFacts['HeartRate'] <= range[1]), runningHealthFacts['duration_sec'], 0)
    list_hr_columns.append(f"HR_{range[0]}-{range[1]}")

In [24]:
list_hr_columns.append('workout_pk')
list_hr_columns.append('duration_sec')

In [25]:
runningHealthFactsHeartrateZones = runningHealthFacts[list_hr_columns].groupby('workout_pk').sum()

In [26]:
runningHealthFactsHeartrateZones

,HR_90-100,HR_100-110,HR_110-120,HR_120-125,HR_125-130,HR_130-135,HR_135-140,HR_140-145,HR_145-150,HR_150-155,...,HR_160-165,HR_165-170,HR_170-175,HR_175-180,HR_180-185,HR_185-190,HR_190-195,HR_195-200,HR_200-250,duration_sec
workout_pk,,,,,,,,,,,,,,,,,,,,,
101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1198.0,0.0,...,1262.0,0.0,676.0,0.0,0.0,0.0,0.0,0.0,0.0,3136.0
112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1184.0,0.0,1186.0,...,0.0,1345.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3715.0
114,0.0,0.0,0.0,0.0,0.0,980.0,0.0,0.0,387.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1367.0
115,0.0,0.0,0.0,0.0,0.0,0.0,1188.0,1187.0,1185.0,311.0,...,1182.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7165.0
116,0.0,0.0,0.0,0.0,0.0,0.0,861.0,0.0,0.0,279.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1732.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,0.0,0.0,0.0,0.0,14.0,4.0,8.0,14.0,10.0,34.0,...,52.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,191.0
591,0.0,59.0,23.0,577.0,1290.0,659.0,84.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2692.0
592,0.0,9.0,54.0,75.0,60.0,178.0,307.0,167.0,137.0,117.0,...,131.0,142.0,119.0,44.0,0.0,0.0,0.0,0.0,0.0,1728.0


In [27]:
for hr_range in list_hr_columns[0:(len(list_hr_columns)-2)]:
    runningHealthFactsHeartrateZones[f'share{hr_range}'] = runningHealthFactsHeartrateZones[f'{hr_range}'] / runningHealthFactsHeartrateZones['duration_sec']

#### Join to Flatfile

In [28]:
runningFactsFlat = runningFactsFlat.merge(runningRoutesFacts, on = 'workout_pk', how = 'left')
runningFactsFlat = runningWorkouts[['workout_pk', 'sourceName', 'workoutActivityType', 'duration']].merge(runningFactsFlat, on = 'workout_pk', how = 'right')
runningFactsFlat = runningFactsFlat.merge(runningHealthFactsHeartrate, on = 'workout_pk', how = 'left')
runningFactsFlat = runningFactsFlat.merge(runningHealthFactsHeartrateZones, on = 'workout_pk', how = 'left')

### Save Flatfile

In [29]:
runningFactsFlat.to_parquet('./output_files/flatfiles/runningFactsFlat.parquet.gzip', compression='gzip')